## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

In [6]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
unique_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [9]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type_counts = application_df["APPLICATION_TYPE"].value_counts()
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [10]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts[app_type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
class_counts = application_df["CLASSIFICATION"].value_counts()

In [25]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C1720        6
C2400        6
C4100        6
C1257        5
C1600        5
C0           3
C1260        3
C2710        3
C1234        2
C1246        2
C1267        2
C3200        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [14]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=["IS_SUCCESSFUL"])
y = application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential() 

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/opt/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=1)

Epoch 1/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 542us/step - accuracy: 0.7049 - loss: 0.5891
Epoch 2/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 573us/step - accuracy: 0.7338 - loss: 0.5516
Epoch 3/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step - accuracy: 0.7324 - loss: 0.5480
Epoch 4/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step - accuracy: 0.7329 - loss: 0.5476
Epoch 5/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step - accuracy: 0.7324 - loss: 0.5481
Epoch 6/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.7321 - loss: 0.5485
Epoch 7/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step - accuracy: 0.7342 - loss: 0.5442
Epoch 8/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 542us/step - accuracy: 0.7307 - loss: 0.5480
Epoch 9/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - accuracy: 0.7328 - loss: 0.5470
Epoch 10/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.7313 - loss: 0.5492
Epoch 11/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - accuracy: 0.7338 - loss: 0.5442
Epoch 12/50
858/858 ━━━━━━━━━━

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 812us/step - accuracy: 0.7227 - loss: 0.5613
Loss: 0.5613309144973755, Accuracy: 0.7227405309677124


In [20]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

In [ ]:
'''STEP 4:
1. Overview

The goal of this analysis is to build a machine learning model
that predicts whether an organization funded by Alphabet Soup
will be successful. Using deep learning techniques, we aim to
analyze various applicant features and determine success rates.
This will help Alphabet Soup allocate resources more effectively.

2. Results

In data preprocessing, the target variable is IS_SUCCESSFUL,
indicating whether an organization’s funding led to success
(1 = successful, 0 = not successful). The features include
APPLICATION_TYPE, AFFILIATION, CLASSIFICATION, USE_CASE,
ORGANIZATION, STATUS, INCOME_AMT, SPECIAL_CONSIDERATIONS,
and ASK_AMT. The EIN and NAME columns were removed since
they are identifiers and do not contribute to the model.
The neural network consists of two hidden layers and one output.
The first hidden layer has 80 neurons with ReLU activation,
while the second has 30 neurons with ReLU. The output layer
has 1 neuron with a Sigmoid activation to predict probabilities.
ReLU improves learning efficiency, while Sigmoid ensures the
output is in a binary classification format.
The initial model did not reach the 75% accuracy target,
requiring further optimization. Several improvements were tried,
including binning rare values, increasing neurons and layers,
testing different activation functions (tanh, LeakyReLU),
increasing training epochs, and adjusting the learning rate.

3. Summary

The model achieved moderate accuracy but did not meet the
75% threshold. Alternative models like Decision Trees,
Random Forest, or XGBoost may yield better results. Feature
engineering, such as creating new variables or interactions,
could also improve accuracy. Hyperparameter tuning via grid
search or random search may further optimize performance.
While the deep learning model provided useful insights,
Random Forest or Gradient Boosting may offer better
interpretability and predictive power.'''